# experiment 5

In [1]:
!pip install kaggle wandb onnx -Uq
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [3]:
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

challenges-in-representation-learning-facial-expression-recognition-challenge.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
! unzip challenges-in-representation-learning-facial-expression-recognition-challenge.zip

Archive:  challenges-in-representation-learning-facial-expression-recognition-challenge.zip
replace example_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: example_submission.csv  
replace fer2013.tar.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: fer2013.tar.gz          
replace icml_face_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: icml_face_data.csv      y
y

replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: train.csv               y



In [7]:
!pip install wandb onnx -Uq

# data staff

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import wandb
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [11]:
wandb.login()

True

In [12]:
class FER2013Dataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        pixels = self.data.iloc[idx]['pixels']
        emotion = self.data.iloc[idx]['emotion']

        image = np.array([int(pixel) for pixel in pixels.split()]).reshape(48, 48)
        image = image.astype(np.float32) / 255.0

        if self.transform:
            image = self.transform(image)
        else:
            image = torch.FloatTensor(image).unsqueeze(0)

        return image, emotion

In [13]:
def get_transforms():
    train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(20),
        transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.3, hue=0.15),
        transforms.RandomAffine(degrees=0, scale=(0.6, 1.4)),
        transforms.RandomErasing(p=0.4, scale=(0.02, 0.2)),
        transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    val_test_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    return train_transform, val_test_transform

In [14]:
def create_train_val_loaders(csv_file, batch_size=64, train_split=0.8):
    train_transform, val_transform = get_transforms()

    full_dataset = FER2013Dataset(csv_file, transform=None)

    train_size = int(train_split * len(full_dataset))
    val_size = len(full_dataset) - train_size

    train_dataset, val_dataset = random_split(
        full_dataset, [train_size, val_size],
        generator=torch.Generator().manual_seed(42)
    )

    train_dataset.dataset.transform = train_transform
    val_dataset.dataset.transform = val_transform

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    return train_loader, val_loader

In [15]:
def create_test_loader(csv_file, batch_size=64):
    _, test_transform = get_transforms()

    test_dataset = FER2013Dataset(csv_file, transform=test_transform)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    return test_loader

In [16]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        self.dropout = nn.Dropout2d(0.15)

    def forward(self, x):
        identity = x

        out = F.relu(self.bn1(self.conv1(x)))
        out = self.dropout(out)
        out = self.bn2(self.conv2(out))

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = F.relu(out)

        return out

In [17]:
class ImprovedFacialExpressionResNet(nn.Module):
    def __init__(self, num_classes=7, dropout_rate=0.6):
        super(ImprovedFacialExpressionResNet, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(32, 64, 2, stride=1)
        self.layer2 = self._make_layer(64, 128, 2, stride=2)
        self.layer3 = self._make_layer(128, 256, 2, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(256, 128)
        self.dropout2 = nn.Dropout(dropout_rate * 0.5)
        self.fc2 = nn.Linear(128, num_classes)

        self._initialize_weights()

    def _make_layer(self, in_channels, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, padding=0, bias=False),
                nn.BatchNorm2d(out_channels)
            )

        layers = []
        layers.append(ResidualBlock(in_channels, out_channels, stride, downsample))

        for _ in range(1, blocks):
            layers.append(ResidualBlock(out_channels, out_channels))

        return nn.Sequential(*layers)

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout1(x)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)

        return x


In [29]:
def test_overfitting():
    print("Testing improved model architecture with overfitting on small dataset...")

    model = ImprovedFacialExpressionResNet(dropout_rate=0.3).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_loader, _ = create_train_val_loaders('train.csv', batch_size=32)

    small_batch = []
    for i, (data, target) in enumerate(train_loader):
        small_batch.extend(list(zip(data, target)))
        if len(small_batch) >= 20:
            break

    small_batch = small_batch[:20]

    for epoch in range(50):
        model.train()
        total_loss = 0
        correct = 0

        for data, target in small_batch:
            data, target = data.unsqueeze(0).to(device), torch.tensor([target]).to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()

        accuracy = 100. * correct / len(small_batch)
        avg_loss = total_loss / len(small_batch)

        print(f"Overfit Epoch {epoch+1}/30, Loss: {avg_loss:.4f}, Acc: {accuracy:.2f}%")

        if accuracy >= 95.0:
            print("Improved model can overfit successfully!")
            break

    print("Overfitting test completed.\n")


In [19]:
def compute_loss(loader, model, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)

            total_loss += loss.item()
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
            total += target.size(0)

    return total_loss / len(loader), 100. * correct / total


In [20]:
def train_model(config=None):
    with wandb.init(project="Facial_Expression_Recognition_5", config=config):
        config = wandb.config

        model = ImprovedFacialExpressionResNet(dropout_rate=config.dropout_rate).to(device)
        train_loader, val_loader = create_train_val_loaders('train.csv', config.batch_size)

        criterion = nn.CrossEntropyLoss(label_smoothing=config.label_smoothing)
        optimizer = optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)

        scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer,
            T_0=len(train_loader) * 5,
            T_mult=1,
            eta_min=config.learning_rate * 0.01
        )

        best_val_acc = 0
        patience_counter = 0

        wandb.watch(model, log="gradients", log_freq=100)

        for epoch in range(config.epochs):
            model.train()
            train_loss = 0
            train_correct = 0
            train_total = 0

            progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{config.epochs} [Train]')

            for batch_idx, (data, target) in enumerate(progress_bar):
                data, target = data.to(device), target.to(device)

                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                loss.backward()

                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)

                optimizer.step()
                scheduler.step()

                train_loss += loss.item()
                pred = output.argmax(dim=1)
                train_correct += pred.eq(target).sum().item()
                train_total += target.size(0)

                if batch_idx % 50 == 0:
                    current_lr = scheduler.get_last_lr()[0]
                    wandb.log({
                        "batch_loss": loss.item(),
                        "learning_rate": current_lr,
                        "epoch": epoch
                    })

                progress_bar.set_postfix({
                    'loss': f'{loss.item():.3f}',
                    'acc': f'{100.*train_correct/train_total:.1f}%'
                })

            train_acc = 100. * train_correct / train_total
            train_loss = train_loss / len(train_loader)

            val_loss, val_acc = compute_loss(val_loader, model, criterion, device)

            wandb.log({
                "epoch": epoch,
                "train_loss": train_loss,
                "train_accuracy": train_acc,
                "val_loss": val_loss,
                "val_accuracy": val_acc,
                "train_val_gap": train_acc - val_acc
            })

            print(f"Epoch {epoch+1}/{config.epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%, Gap: {train_acc-val_acc:.2f}%")

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                patience_counter = 0

                model_artifact = wandb.Artifact(
                    name=f"best_model_{wandb.run.name}",
                    type="model",
                    description=f"Best improved ResNet model with validation accuracy: {val_acc:.2f}%"
                )

                model_save_dict = {
                    'model_state_dict': model.state_dict(),
                    'model_config': {
                        'num_classes': 7,
                        'dropout_rate': config.dropout_rate
                    },
                    'training_config': dict(config),
                    'val_accuracy': val_acc,
                    'epoch': epoch,
                    'model_architecture': 'ImprovedFacialExpressionResNet'
                }

                torch.save(model_save_dict, "best_model.pth")

                model_artifact.add_file("best_model.pth")
                wandb.log_artifact(model_artifact)

                print(f"New best model saved with validation accuracy: {val_acc:.2f}%")
            else:
                patience_counter += 1

            if patience_counter >= config.patience:
                print(f"Early stopping triggered after {epoch+1} epochs")
                break

        wandb.log({
            "best_val_accuracy": best_val_acc
        })

        print(f"Training completed. Best validation accuracy: {best_val_acc:.2f}%")

        return best_val_acc


In [21]:
def evaluate_model_on_testset(model_artifact_path, test_csv_file, batch_size=64):
    checkpoint = torch.load(model_artifact_path, map_location=device)

    model_config = checkpoint['model_config']
    model = ImprovedFacialExpressionResNet(
        num_classes=model_config['num_classes'],
        dropout_rate=model_config['dropout_rate']
    ).to(device)

    model.load_state_dict(checkpoint['model_state_dict'])

    test_loader = create_test_loader(test_csv_file, batch_size)

    criterion = nn.CrossEntropyLoss()
    test_loss, test_acc = compute_loss(test_loader, model, criterion, device)

    print(f"Test Accuracy: {test_acc:.2f}%")
    print(f"Test Loss: {test_loss:.4f}")

    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            all_preds.extend(pred.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

    print("\nClassification Report:")
    print(classification_report(all_targets, all_preds, target_names=emotion_labels))

    cm = confusion_matrix(all_targets, all_preds)

    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=emotion_labels, yticklabels=emotion_labels)
    plt.title('Confusion Matrix - Facial Expression Recognition (Improved ResNet)')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()

    return test_acc, all_preds, all_targets


In [31]:
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'best_val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'learning_rate': {
            'distribution': 'log_uniform_values',
            'min': 0.00005,
            'max': 0.005
        },
        'batch_size': {
            'values': [32, 64]
        },
        'dropout_rate': {
            'distribution': 'uniform',
            'min': 0.4,
            'max': 0.7
        },
        'weight_decay': {
            'distribution': 'log_uniform_values',
            'min': 1e-4,
            'max': 1e-2
        },
        'label_smoothing': {
            'distribution': 'uniform',
            'min': 0.1,
            'max': 0.3
        },
        'epochs': {
            'value': 30
        },
        'patience': {
            'value': 8
        }
    }
}

In [32]:
test_overfitting()

Testing improved model architecture with overfitting on small dataset...
Overfit Epoch 1/30, Loss: 1.9576, Acc: 25.00%
Overfit Epoch 2/30, Loss: 1.8440, Acc: 30.00%
Overfit Epoch 3/30, Loss: 1.7916, Acc: 30.00%
Overfit Epoch 4/30, Loss: 1.7272, Acc: 30.00%
Overfit Epoch 5/30, Loss: 1.6530, Acc: 30.00%
Overfit Epoch 6/30, Loss: 1.5868, Acc: 30.00%
Overfit Epoch 7/30, Loss: 1.5151, Acc: 30.00%
Overfit Epoch 8/30, Loss: 1.4373, Acc: 30.00%
Overfit Epoch 9/30, Loss: 1.3532, Acc: 35.00%
Overfit Epoch 10/30, Loss: 1.2424, Acc: 35.00%
Overfit Epoch 11/30, Loss: 1.1876, Acc: 35.00%
Overfit Epoch 12/30, Loss: 1.1600, Acc: 50.00%
Overfit Epoch 13/30, Loss: 1.0732, Acc: 50.00%
Overfit Epoch 14/30, Loss: 1.0140, Acc: 50.00%
Overfit Epoch 15/30, Loss: 0.9429, Acc: 55.00%
Overfit Epoch 16/30, Loss: 0.9485, Acc: 55.00%
Overfit Epoch 17/30, Loss: 0.8998, Acc: 45.00%
Overfit Epoch 18/30, Loss: 0.8409, Acc: 65.00%
Overfit Epoch 19/30, Loss: 0.8472, Acc: 60.00%
Overfit Epoch 20/30, Loss: 0.8300, Acc: 50.

In [33]:
sweep_id = wandb.sweep(sweep_config, project="Facial_Expression_Recognition_5")

Create sweep with ID: als3wxaz
Sweep URL: https://wandb.ai/konstantine25b-free-university-of-tbilisi-/Facial_Expression_Recognition_5/sweeps/als3wxaz


In [34]:
print("Running improved hyperparameter sweep with anti-overfitting ResNet...")
wandb.agent(sweep_id, train_model, count=3)

Running improved hyperparameter sweep with anti-overfitting ResNet...


wandb: Agent Starting Run: 1bdep6nf with config:
wandb: 	batch_size: 32
wandb: 	dropout_rate: 0.5479010231390392
wandb: 	epochs: 30
wandb: 	label_smoothing: 0.22223961117304525
wandb: 	learning_rate: 0.0003507942900207464
wandb: 	patience: 8
wandb: 	weight_decay: 0.0007072641496874819


Epoch 1/30 [Train]: 100%|██████████| 718/718 [00:32<00:00, 22.03it/s, loss=1.654, acc=34.6%]


Epoch 1/30, Train Loss: 1.7682, Train Acc: 34.64%, Val Loss: 1.6647, Val Acc: 42.51%, Gap: -7.87%
New best model saved with validation accuracy: 42.51%


Epoch 2/30 [Train]: 100%|██████████| 718/718 [00:31<00:00, 22.46it/s, loss=1.463, acc=44.6%]


...

Epoch 29/30 [Train]: 100%|██████████| 718/718 [00:32<00:00, 22.43it/s, loss=1.099, acc=85.2%]


Epoch 29/30, Train Loss: 1.1119, Train Acc: 85.25%, Val Loss: 1.6087, Val Acc: 55.96%, Gap: 29.29%
New best model saved with validation accuracy: 55.96%


Epoch 30/30 [Train]: 100%|██████████| 718/718 [00:31<00:00, 22.46it/s, loss=1.053, acc=87.5%]


Epoch 30/30, Train Loss: 1.0813, Train Acc: 87.51%, Val Loss: 1.6137, Val Acc: 55.59%, Gap: 31.92%
Training completed. Best validation accuracy: 55.96%


batch_loss,▇▆▆▇▆▇▆█▆▅▅▆▅▅▅▅▆▆▆▄▅▄▄▄▅▃▄▁▃▄▃▃▄▄▃▃▂▃▁▁
best_val_accuracy,▁
epoch,▁▁▁▁▁▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇█████
learning_rate,███▇▄▁█▇▇▆▅▄▄▂▂▁▁▇▇▆▃▂███▅▄▄▁▁▆▅▄▂█▆▄▃▂▁
train_accuracy,▁▂▃▃▃▃▃▄▄▄▄▄▅▅▆▅▅▆▆▇▆▆▇▇▇▇▇▇██
train_loss,█▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▃▂▂▁▁
train_val_gap,▁▂▃▃▃▃▃▃▄▄▃▄▄▅▅▄▅▅▆▆▆▆▆▇▇▆▇▇██
val_accuracy,▁▂▃▄▄▄▅▆▆▆▆▆▇▇▇▇▇▇██▇▇███▇█▇██
val_loss,█▅▄▃▃▄▂▂▂▁▂▁▁▁▁▁▁▂▂▂▃▂▂▃▃▃▃▄▄▄
batch_loss,1.01189
best_val_accuracy,55.95611


In [35]:
api = wandb.Api()
runs = api.runs("konstantine25b-free-university-of-tbilisi-/Facial_Expression_Recognition_5")

best_run = None
best_val_acc = 0

In [36]:
for run in runs:
    if run.state == "finished" and "best_val_accuracy" in run.summary:
        val_acc = run.summary["best_val_accuracy"]
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_run = run

In [38]:
if best_run:
    print(f"\nBest run: {best_run.name}")
    print(f"Best validation accuracy: {best_val_acc:.2f}%")
    print(f"Best hyperparameters: {best_run.config}")

    # Download the artifact
    artifacts = api.artifact(f"konstantine25b-free-university-of-tbilisi-/Facial_Expression_Recognition_5/best_model_{best_run.name}:latest")
    download_path = artifacts.download()

    # Check what was actually downloaded
    print(f"Downloaded to: {download_path}")

    # Find the correct path to the model file
    import os
    model_file_path = None

    # Check in the download directory
    if os.path.exists(os.path.join(download_path, "best_model.pth")):
        model_file_path = os.path.join(download_path, "best_model.pth")
    # Check in current directory
    elif os.path.exists("best_model.pth"):
        model_file_path = "best_model.pth"
    # Search recursively
    else:
        for root, dirs, files in os.walk("."):
            if "best_model.pth" in files:
                model_file_path = os.path.join(root, "best_model.pth")
                break

    if model_file_path and os.path.exists(model_file_path):
        print(f"Found model file at: {model_file_path}")

        # Create final artifact with correct path
        final_artifact = wandb.Artifact(
            name="final_best_resnet_model",
            type="model",
            description=f"Final best ResNet model with {best_val_acc:.2f}% validation accuracy"
        )
        final_artifact.add_file(model_file_path)

        # Upload the final artifact
        with wandb.init(project="Facial_Expression_Recognition_5", name="final_model_upload"):
            wandb.log_artifact(final_artifact)
            wandb.log({
                "final_best_val_accuracy": best_val_acc,
                "model_architecture": "ResNet",
                "ready_for_testing": True
            })

        print(f"\nFinal model uploaded to wandb as 'final_best_resnet_model'")
        print("You can now load this model in the future for testing on any dataset!")

        print("\nTo test on a different dataset in the future, use:")
        print("evaluate_model_on_testset('path_to_downloaded_model.pth', 'your_test_dataset.csv')")

    else:
        print("Error: Could not find the downloaded model file!")
        print("Files in current directory:")
        for item in os.listdir("."):
            print(f"  {item}")

else:
    print("No successful runs found!")


Best run: lyric-sweep-1
Best validation accuracy: 57.65%
Best hyperparameters: {'epochs': 30, 'patience': 8, 'batch_size': 32, 'dropout_rate': 0.5479010231390392, 'weight_decay': 0.0007072641496874819, 'learning_rate': 0.0003507942900207464, 'label_smoothing': 0.22223961117304525}


wandb:   1 of 1 files downloaded.  


Downloaded to: /content/artifacts/best_model_lyric-sweep-1:v10
Found model file at: /content/artifacts/best_model_lyric-sweep-1:v10/best_model.pth


final_best_val_accuracy,▁
final_best_val_accuracy,57.64542
model_architecture,ResNet
ready_for_testing,True



Final model uploaded to wandb as 'final_best_resnet_model'
You can now load this model in the future for testing on any dataset!

To test on a different dataset in the future, use:
evaluate_model_on_testset('path_to_downloaded_model.pth', 'your_test_dataset.csv')
